# 7장 네이버 뉴스 요약 프로젝트 

## 7.2 사전 준비

- !pip install gensim BeautifulSoup4 lxml requests

## 7.3 사전 지식 쌓기

## 7.3.1 User-Agent 확인하기
- www.useragentstring.com

In [ ]:
# 해당 분야 상위 뉴스 HTML 가져오기
headers = {'User-Agent' : '<복사한 user-agent 값 대체>'} # ex. 'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) ...생략...'

res = requests.get(news_link, headers = headers)
print(res.text)

## 7.3.2 네이버 뉴스 구조 이해하기–섹션별 접속 주소(URL) 확인

#### 정치 섹션 url
- https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100

#### 경제 섹션 url
- https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=101

#### 사회 섹션 url
- https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=102

In [ ]:
for sid in ['100', '101', '102']:
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
                + "&sid1=" \
                + sid \
             
    print("section url : ", sec_url)

## 7.3.3 네이버 뉴스 구조 이해하기–상위 랭킹 세개의 뉴스 메타 정보 확인

In [ ]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {'User-Agent' : '<복사한 user-agent 값 대체>'}
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')

    return soup

default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"
for sid in ['100']:#, '101', '102']:
    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
              + "&sid1=" \
              + sid
    print("section url : ", sec_url)

    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)

    # 해당 분야 상위 뉴스 3개 가져오기
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") ,
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" : li.img.attrs.get('src') if li.img else default_img
        }
        print(news_info)

## 7.3.4 gensim으로 뉴스 요약하기
-  gensim 라이브러리 설명 : https://radimrehurek.com/gensim
- summarize(): https://radimrehurek.com/gensim_3.8.3/summarization/summariser.html

# 7.4 구현

## Step1) 네이버 뉴스 크롤링하기

In [ ]:
import requests
from bs4 import BeautifulSoup
import bs4.element
import datetime

# BeautifulSoup 객체 생성
def get_soup_obj(url):
    headers = {'User-Agent' : '<복사한 user-agent 값 대체>'}
    res = requests.get(url, headers = headers)
    soup = BeautifulSoup(res.text,'lxml')

    return soup


# 뉴스의 기본 정보 가져오기
def get_top3_news_info(sec, sid):
    # 임시 이미지
    default_img = "https://search.naver.com/search.naver?where=image&sm=tab_jum&query=naver#"

    # 해당 분야 상위 뉴스 목록 주소
    sec_url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec" \
              + "&sid1=" \
              + sid
    print("section url : ", sec_url)

    # 해당 분야 상위 뉴스 HTML 가져오기
    soup = get_soup_obj(sec_url)

    # 해당 분야 상위 뉴스 3개 가져오기
    news_list3 = []
    lis3 = soup.find('ul', class_='type06_headline').find_all("li", limit=3)
    for li in lis3:
        # title : 뉴스 제목, news_url : 뉴스 URL, image_url : 이미지 URL
        news_info = {
            "title" : li.img.attrs.get('alt') if li.img else li.a.text.replace("\n", "").replace("\t","").replace("\r","") ,
            "date" : li.find(class_="date").text,
            "news_url" : li.a.attrs.get('href'),
            "image_url" : li.img.attrs.get('src') if li.img else default_img
        }
        news_list3.append(news_info)

    return news_list3

# 뉴스 본문 가져오기
def get_news_contents(url):
    soup = get_soup_obj(url)
    body = soup.find('div', class_="_article_body_contents")

    news_contents = ''
    for content in body:
        if type(content) is bs4.element.NavigableString and len(content) > 50:
            # content.strip() : whitepace 제거 (참고 : https://www.tutorialspoint.com/python3/string_strip.htm)
            # 뉴스 요약을 위하여 '.' 마침표 뒤에 한칸을 띄워 문장을 구분하도록 함
            news_contents += content.strip() + ' '

    return news_contents


# '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
def get_naver_news_top3():
    # 뉴스 결과를 담아낼 dictionary
    news_dic = dict()

    # sections : '정치', '경제', '사회'
    sections = ["pol", "eco","soc"]
    # section_ids : URL에 사용될 뉴스 각 부문 ID
    section_ids = ["100", "101","102"]

    for sec, sid in zip(sections, section_ids):
        # 뉴스의 기본 정보 가져오기
        news_info = get_top3_news_info(sec, sid)
        #print(news_info)
        for news in news_info:
            # 뉴스 본문 가져오기
            news_url = news['news_url']
            news_contents = get_news_contents(news_url)

            # 뉴스 정보를 저장하는 dictionary를 구성
            news['news_contents'] = news_contents

        news_dic[sec] = news_info

    return news_dic

# 함수 호출 - '정치', '경제', '사회' 분야의 상위 3개 뉴스 크롤링
news_dic = get_naver_news_top3()
# 경제의 첫번째 결과 확인하기
news_dic['eco'][0]

## Step2) 뉴스 요약하기

In [ ]:
from gensim.summarization.summarizer import summarize

# 섹션 지정
my_section = 'eco'
news_list3 = news_dic[my_section]
# 뉴스 요약하기
for news_info in news_list3:
    # 뉴스 본문이 10 문장 이하일 경우 결과가 반환되지 않음.
    # 이때는 요약하지 않고 본문에서 앞 3문장을 사용함.
    try:
        snews_contents = summarize(news_info['news_contents'], word_count=20)
    except:
        snews_contents = None

    if not snews_contents:
        news_sentences = news_info['news_contents'].split('.')

        if len(news_sentences) > 3:
            snews_contents = '.'.join(news_sentences[:3])
        else:
            snews_contents = '.'.join(news_sentences)

    news_info['snews_contents'] = snews_contents

## 요약 결과 - 첫번째 뉴스
print("==== 첫번째 뉴스 원문 ====")
print(news_list3[0]['news_contents'])
print("\n==== 첫번째 뉴스 요약문 ====")
print(news_list3[0]['snews_contents'])

## 요약 결과 - 두번째 뉴스
print("==== 두번째 뉴스 원문 ====")
print(news_list3[1]['news_contents'])
print("\n==== 두번째 뉴스 요약문 ====")
print(news_list3[1]['snews_contents'])

## Step3) 카카오 메시지 보내기 위한 사전 준비하기

In [ ]:
import json
import kakao_utils

#token이 저장된 파일
KAKAO_TOKEN_FILENAME = "res/kakao_message/kakao_token.json"
KAKAO_APP_KEY = "<REST_API 앱키를 입력하세요>"
kakao_utils.update_tokens(KAKAO_APP_KEY, KAKAO_TOKEN_FILENAME)

## Step4) 리스트 템플릿으로 뉴스 목록 전송하기

In [ ]:
# 사용자가 선택한 카테고리를 제목에 넣기 위한 dictionary
sections_ko = {'pol': '정치', 'eco' : '경제', 'soc' : '사회'}

# 네이버 뉴스 URL
navernews_url = "https://news.naver.com/main/home.nhn"

# 추후 각 리스트에 들어갈 내용(content) 만들기
contents = []

# 리스트 템플릿 형식 만들기
template = {
    "object_type" : "list",
    "header_title" : sections_ko[my_section] + " 분야 상위 뉴스 빅3",
    "header_link" : {
        "web_url": navernews_url,
        "mobile_web_url" : navernews_url
    },
    "contents" : contents,
    "button_title" : "네이버 뉴스 바로가기"
}
## 내용 만들기
# 각 리스트에 들어갈 내용(content) 만들기
for news_info in news_list3:
    content = {
        "title" : news_info.get('title'),
        "description" : "작성일 : " + news_info.get('date'),
        "image_url" : news_info.get('image_url'),
        "image_width" : 50, "image_height" : 50,
        "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
        }
    }

    contents.append(content)

# 카카오톡 메시지 전송
res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
if res.json().get('result_code') == 0:
    print('뉴스를 성공적으로 보냈습니다.')
else:
    print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())

## Step5) 텍스트 템플릿으로 뉴스 요약본 전송하기

In [ ]:
# 3번에 걸쳐 각 뉴스의 요약 결과를 전송합니다.
for idx, news_info in enumerate(news_list3):
    # 텍스트 템플릿 형식 만들기
    template = {
        "object_type": "text",
        "text": '# 제목 : ' + news_info.get('title') + \
                '\n\n# 요약 : ' + news_info.get('snews_contents'),
        "link": {
            "web_url": news_info.get('news_url'),
            "mobile_web_url": news_info.get('news_url')
        },
        "button_title": "자세히 보기"
    }
    
    # 카카오톡 메시지 전송
    res = kakao_utils.send_message(KAKAO_TOKEN_FILENAME, template)
    if res.json().get('result_code') == 0:
        print('뉴스를 성공적으로 보냈습니다.')
    else:
        print('뉴스를 성공적으로 보내지 못했습니다. 오류메시지 : ', res.json())